<a href="https://colab.research.google.com/github/Minwoo-study/Naver_map_rating/blob/main/NAVER_MAP_CRAWLING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 지도 - 식당 리뷰 크롤링

###### step0. Preparation <br>
###### step1. 상점 리스트 크롤링 <br>
###### step2. 리뷰 크롤링 <br>

- jupyter notebook으로 실행하세요
- 2021년 10월 26일부로 네이버 리뷰 별점 제도가 폐지되어, 전체 리뷰 텍스트와 별점 있는 리뷰를 따로 다룹니다.


## Preparation

In [ ]:
pip install selenium #selenium 설치

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException #except문 처리
import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

## Step1. 상점 리스트 크롤링

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window() # 창 최대화

# 네이버 지도 url
url = 'https://map.naver.com/v5/search'
driver.get(url)

time.sleep(3)

# 키워드 검색하기
keyword = "부평 맛집"

search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)

# 결과 뜰 때까지 기다리기
try: 
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.input_box>input.input_search")))
except:
    driver.quit()

In [ ]:
# iframe(html 안에 html) 생성
driver.switch_to.frame("searchIframe") # 다시 전체 페이지로 빠져나오기 : switch_to_default_content()

### 상점 리스트  <br>
- 60곳 추출

In [ ]:
title_list = []
data_list = []

try: 
    for i in range(1,7): # 페이지
        driver.find_element(By.LINK_TEXT,str(i)).click()
        try: 
            for j in range(3,70,3): 
                element = driver.find_elements(By.CSS_SELECTOR,'.OXiLu')[j]
                ActionChains(driver).move_to_element(element).key_down(Keys.PAGE_DOWN).perform()
        except:
            pass
        
        # 상점 이름
        title_raw = driver.find_elements(By.CSS_SELECTOR,".OXiLu")
        for title in title_raw:
            title = title.text
            title_list.append(title)

        # 영업 정보, 별점, 리뷰 개수 등 데이터
        data_raw = driver.find_elements(By.CSS_SELECTOR,'._17H46')
        for data in data_raw: 
            data = data.text
            data_list.append(data)
        
except:
    pass

print(len(title_list),len(data_list))

60 60


In [ ]:
# 데이터셋 만들기
df = pd.DataFrame({'title':title_list, 'data':data_list, 'map':len(data_list)*""}) # map: 식당 url 저장
df.drop_duplicates() # 중복된 값 제거
df

,title,data,map
0,착한쭝식 부평시장역점,"영업 종료\n별점\n4.13방문자리뷰 1,093블로그리뷰 79",
1,타이베이101우육면 본점,영업 종료\n별점\n4.55방문자리뷰 454블로그리뷰 174,
2,북새통,영업 종료\n별점\n4.37방문자리뷰 757블로그리뷰 287,
3,뚝배기이탈리아,영업 종료\n별점\n4.5방문자리뷰 673블로그리뷰 238,
4,오구당당 부평본점,"영업 종료\n별점\n4.38방문자리뷰 1,962블로그리뷰 867",
5,그날곱창,"곧 영업 종료\n별점\n4.88방문자리뷰 1,041블로그리뷰 242",
6,쭈꾸미도사 부평삼산점,곧 영업 종료\n별점\n4.48방문자리뷰 368블로그리뷰 88,
7,오캄,영업 종료\n별점\n4.5방문자리뷰 685블로그리뷰 712,
8,피플즈,"영업 종료\n별점\n4.56방문자리뷰 1,209블로그리뷰 1,144",
9,부평막국수,영업 종료\n별점\n4.36방문자리뷰 707블로그리뷰 275,


In [ ]:
for i, keyword in enumerate(df['title'].tolist()): 
    
    try: 
        keyword += ' 부평' # 식당 이름에 keyword(부평) 추가, 정확도 높이기 위함
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 
        driver.get(naver_map_search_url) # url 접속
        time.sleep(4) 
        
        res_code = re.findall(r"place/(\d+)", driver.current_url) 
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' # 리뷰 크롤링 위한 url
        df['map'][i] = final_url 
        time.sleep(2)
        
    except: # 검색 결과가 없을 때 / 결과 여러 개 / 식당 탭으로 바로 연결되지 않을 때 
        df['map'][i]= '' 

In [ ]:
#데이터셋 저장
df.to_csv('부평_data_with_url.csv', encoding = 'utf-8-sig') # 실행할 때마다 결과값이 조금씩 달라질 수 있음

In [ ]:
# 현재 탭 종료
driver.close()

## Step2. 리뷰 크롤링

In [ ]:
# 데이터 불러오기
df= pd.read_csv('부평_data_with_url.csv', index_col=0)

In [ ]:
# map url 없는 식당 제외
df = df.dropna(axis = 0) 
df = df.reset_index(drop=True)

In [ ]:
df

,title,data,map
0,타이베이101우육면 본점,영업 종료\n별점\n4.55방문자리뷰 454블로그리뷰 174,https://pcmap.place.naver.com/restaurant/17830...
1,오구당당 부평본점,"영업 종료\n별점\n4.38방문자리뷰 1,962블로그리뷰 867",https://pcmap.place.naver.com/restaurant/11719...
2,그날곱창,"곧 영업 종료\n별점\n4.88방문자리뷰 1,041블로그리뷰 242",https://pcmap.place.naver.com/restaurant/12309...
3,오캄,영업 종료\n별점\n4.5방문자리뷰 685블로그리뷰 712,https://pcmap.place.naver.com/restaurant/11415...
4,누룩골 삼산점,"영업 종료\n별점\n4.51방문자리뷰 2,241블로그리뷰 222",https://pcmap.place.naver.com/restaurant/10303...
5,구도로통닭 부평점,영업 중\n별점\n4.46방문자리뷰 391블로그리뷰 123,https://pcmap.place.naver.com/restaurant/12397...
6,브로이인탭하우스,영업 종료\n별점\n4.74방문자리뷰 44블로그리뷰 223,https://pcmap.place.naver.com/restaurant/34698...
7,면박사가야밀면,영업 종료\n별점\n4.46방문자리뷰 547블로그리뷰 124,https://pcmap.place.naver.com/restaurant/19990...
8,에키노마에,"영업 종료\n별점\n4.48방문자리뷰 3,587블로그리뷰 477",https://pcmap.place.naver.com/restaurant/37392...
9,뉴욕반점,"영업 종료방문자리뷰 1,122블로그리뷰 595",https://pcmap.place.naver.com/restaurant/20063...


### 크롤러

In [ ]:
def crawling(index):
    
    driver = webdriver.Chrome()
    driver.maximize_window() # 창 최대화

    # 리뷰 url
    url = df['map'][index]
    driver.get(url)
    time.sleep(2)

    # 스크롤 내리기
    if driver.find_element(By.CLASS_NAME,"place_section_count").text < str(30): #리뷰 개수 ~ 30개 
        driver.execute_script("window.scrollTo(0,680)") # 680(위치)만큼 스크롤 내리기
    else:
        driver.execute_script("window.scrollTo(0,1080)") 
    time.sleep(0.5)
    
    # 최신순으로 정렬(클릭)
    try:
        driver.find_element(By.XPATH, "//*[@id='app-root']/div/div/div/div[7]/div[2]/div[3]/div[1]/div[3]/div/span[2]/a").click() 
    except NoSuchElementException:
        driver.find_element(By.XPATH, "//*[@id='app-root']/div/div/div/div[7]/div[2]/div[3]/div[1]/div[4]/div/span[2]/a").click()
    except NoSuchElementException: 
        driver.find_element(By.XPATH, "//*[@id='app-root']/div/div/div/div[7]/div[2]/div[2]/div/div[3]/div/span[2]/a").click()    
    except:
        driver.find_elements(By.CSS_SELECTOR,'_1sWJ8').click()
    time.sleep(1)

    # 스크롤 내려서 리뷰 더보기 클릭하기
    while True:
        driver.execute_script("window.scrollTo(0,3080)")
        try:
            elem = driver.find_element(By.XPATH,"//*[@id='app-root']/div/div/div/div[7]/div[2]/div[3]/div[2]/a") 
            elem.click() # 더보기 클릭
            time.sleep(2)
        except:
            break

    # beautifulsoup으로 분석
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    texts = [] #전체 리뷰 텍스트
    dates = [] # 전체 날짜
    rating_text = [] # 별점 있는 리뷰 텍스트
    rating_date = [] # 별점 있는 날짜
    rating = [] # 별점

    boxes = soup.find_all("li", attrs={"class":re.compile("_3l2Wz")}) # 전체 review boxes

    for box in boxes:
        review = box.find("span", attrs={'class':'WoYOw'}) # 리뷰 텍스트
        rate = box.find("em") # 별점
        date = box.find("time").get_text() # 작성 날짜

        if review:
            review = review.get_text()
            texts.append(review)
            dates.append(date)
            if rate: # 별점이 있는 경우
                rate = rate.get_text()
                rating.append(rate)
                rating_text.append(review)
                rating_date.append(date)
        else:
            continue # 리뷰 텍스트 없으면 제외

    # df 만들기
    total = pd.DataFrame({'날짜':dates, '리뷰':texts})
    total.drop_duplicates(['날짜','리뷰']) # 중복된 값(전체 행) 제거

    rateVer = pd.DataFrame({'날짜':rating_date, '리뷰':rating_text, '평점':rating})
    rateVer.drop_duplicates(['날짜','리뷰','평점']) 
    
    print(f"{index} 끝")
    
    return total, rateVer

### 크롤링
- 중간에 에러 발생시, 새로 df_total, df_rate를 정의하지 않고 끊어서 실행한 후 concat하세요

In [ ]:
df_total = pd.DataFrame()
df_rate = pd.DataFrame()

for i in range(len(df)):
    if df['map'][i] != None : 
        total, rate = crawling(i)
        df_total = pd.concat([df_total, total], ignore_index=True) # 식당별 리뷰 합치기
        df_rate = pd.concat([df_rate, rate], ignore_index=True)
    else:
        continue # map 없는 장소는 제외

0 끝
1 끝
2 끝
3 끝
4 끝
5 끝
6 끝
7 끝
8 끝
9 끝
10 끝
11 끝
12 끝
13 끝
14 끝
15 끝
16 끝
17 끝
18 끝
19 끝
20 끝
21 끝
22 끝
23 끝
24 끝
25 끝
26 끝
27 끝
28 끝
29 끝
30 끝
31 끝
32 끝
33 끝
34 끝
35 끝
36 끝
37 끝
38 끝
39 끝


In [ ]:
# csv로 저장
df_total.to_csv(f'total-review-부평.csv', encoding='utf-8-sig')
df_rate.to_csv(f'rate-review-부평.csv', encoding='utf-8-sig')

In [ ]:
# 전체 종료
driver.quit() 